In [ ]:
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.remote.webelement import WebElement

import cv2
import os
import time

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import sys
sys.path.insert(0, '..')

from web_interaction.main import open_game, start_game, wait_loading, insert_user_stories_from_image
from web_interaction.main import fill_game_main_info_from_image
from web_interaction.main import apply_decompose_action, apply_user_story_action
from game.game import ProductOwnerGame
from game.userstory_card.userstory_card_info import UserStoryCardInfo
from environment.backlog_env import BacklogEnv
from environment.environment import ProductOwnerEnv
from environment.reward_sytem.base_reward_system import BaseRewardSystem
from environment.userstory_env import UserstoryEnv
import image_parser

from pipeline.study_agent import load_dqn_agent
from single_color_storage import SingleColorStorage

In [ ]:
driver = open_game()

iframe = driver.find_element(by=By.ID, value='game_drop')
height = iframe.rect["height"]  # 540
width = iframe.rect["width"]  # 960

wait_loading(iframe)

start_game(driver, iframe)

In [ ]:
iframe.screenshot('iframe.png')

image = cv2.imread('iframe.png')

In [ ]:
class EmptyColorStorage:
    def __init__(self) -> None:
        pass

    def get_unused_color(self, uc_type):
        pass

    def release_color(self, us_type, color):
        pass

In [ ]:
game = ProductOwnerGame()
context = game.context

context.is_new_game = False
game.is_first_release = False
game.userstories.disable_restrictions()
game.office.toggle_purchases(True)

available_developers_count = 2
current_rooms_counter = 1
blank_sprint_counter = 0

game.context.color_storage = EmptyColorStorage()

In [ ]:
insert_user_stories_from_image(game, image)

game.userstories.stories_list

In [ ]:
fill_game_main_info_from_image(game, image)

print('Sprint', game.context.current_sprint)
print('Sprint hours', game.context.current_sprint_hours)
print('Money', game.context.get_money())
print('Loyalty', game.context.get_loyalty())
print('Customers', game.context.customers)
print('Credit', game.context.credit)

In [ ]:
userstory_env = UserstoryEnv(2, 0, 0)
backlog_env = BacklogEnv(6, 0, 0, 0, 0, 0)
reward_system = BaseRewardSystem(config={})
env = ProductOwnerEnv(userstory_env, backlog_env, with_info=True, reward_system=reward_system)
env.game = game

In [ ]:
agent = load_dqn_agent('../models/credit_start_model.pt')

In [ ]:
from game.backlog_card.backlog_card import Card
from game.backlog_card.card_info import CardInfo
from web_interaction.main import (
    click_on_element,
    select_backlog_board,
    click_board_button,
    apply_start_sprint_action,
    select_user_story_board,
)


def find_backlog_card_position(card: Card, image: cv2.typing.MatLike):
    card: CardInfo = card.info
    backlog_cards = image_parser.get_backlog(image)
    for color, hours, position in backlog_cards:
        if color != card.color:
            continue
        if hours != card.hours:
            continue
        return position


def apply_backlog_card_action(
    action: int, driver, iframe: WebElement, env: ProductOwnerEnv
):
    print("Start moving backlog card")
    select_backlog_board(driver, iframe, width, height)

    time.sleep(2)

    filename = "backlog_board.png"
    iframe.screenshot(filename)
    backlog_board_image = cv2.imread(filename)
    # os.remove(filename)

    card = env.backlog_env.get_card(action)
    print("Selected card", card)
    position = find_backlog_card_position(card, backlog_board_image)
    print("Found at position", position)

    click_on_element(driver, iframe, *position)

    print("Clicked on card")

    env._perform_action_backlog_card(action)


def apply_release_action(
    driver, iframe: WebElement, width: int, height: int, env: ProductOwnerEnv
):
    click_on_element(driver, iframe, 115, 455)

    time.sleep(1)

    filename = "game_state.png"
    iframe.screenshot(filename)
    game_image = cv2.imread(filename)
    # os.remove(filename)

    fill_game_main_info_from_image(env.game, game_image)

    env._perform_release()


def apply_buy_statistical_research_action(
    driver, iframe: WebElement, env: ProductOwnerEnv
):
    select_user_story_board(driver, iframe, width, height)
    time.sleep(2)

    click_on_element(driver, iframe, 765, 115)
    time.sleep(2)

    env._perform_statistical_research()

    filename = "user_stroy.png"
    iframe.screenshot(filename)
    userstory_image = cv2.imread(filename)
    # os.remove(filename)

    insert_user_stories_from_image(game, userstory_image)


def apply_web_action(action: int, driver, iframe: WebElement, env: ProductOwnerEnv):
    if action == 0:  # start sprint

        apply_start_sprint_action(driver, iframe, width, height, env)

        return

    if action == 1:  # decompose

        apply_decompose_action(driver, iframe, width, height, env)

        return

    if action == 2:  # release

        apply_release_action(driver, iframe, width, height, env)

        return

    if action == 5:  # buy statistical research

        apply_buy_statistical_research_action(driver, iframe, env)

        return

    if action >= env.meta_action_dim:

        action -= env.meta_action_dim

    if action < env.userstory_env.max_action_num:

        apply_user_story_action(action, driver, iframe, env)

        return

    action -= env.userstory_env.max_action_num

    if action < env.backlog_env.backlog_max_action_num:

        apply_backlog_card_action(action, driver, iframe, env)

        return

    raise Exception(f"Unknown action: {action}")

In [ ]:
state = env._get_state()
print(state)

info = env.get_info()
print(info)

action = agent.get_action(state, info)
print(action)

In [ ]:
apply_web_action(action, driver, iframe, env)

In [ ]:
iframe.rect

In [ ]:
driver.quit()